In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
data_path = "./dane/"
df = pd.read_excel(os.path.join(data_path, "dane3.xlsx"))
df.describe()

,Zadanie,M1,M2,M3,M4,M5,M6,M7,M8,M9,...,M11,M12,M13,M14,M15,M16,M17,M18,M19,M20
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.00000,200.000000,200.000000,200.0000,...,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.00000,200.000000,200.000000,200.000000
mean,100.500000,50.555000,50.050000,51.670000,52.640000,46.010000,49.71500,51.165000,51.595000,49.6950,...,50.160000,48.350000,50.475000,51.690000,49.040000,53.210000,51.18500,51.190000,49.525000,48.565000
std,57.879185,28.564077,28.280851,28.673191,26.607333,28.039812,29.62638,28.684522,29.177519,28.7951,...,28.637299,28.449329,27.032818,27.968286,29.744446,29.322088,28.77378,28.450678,29.748179,27.599355
min,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,2.00000,1.000000,1.000000,1.0000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000
25%,50.750000,27.750000,28.000000,28.000000,29.000000,23.750000,24.00000,26.000000,28.000000,27.0000,...,26.750000,24.000000,29.750000,28.000000,22.000000,26.000000,26.75000,26.750000,24.000000,22.000000
50%,100.500000,50.000000,52.000000,52.000000,53.000000,42.000000,48.50000,56.500000,50.000000,50.5000,...,50.500000,45.500000,51.000000,54.000000,50.000000,56.500000,52.50000,53.500000,47.500000,51.000000
75%,150.250000,73.250000,74.000000,76.250000,75.250000,66.000000,73.00000,74.000000,78.000000,75.2500,...,74.250000,73.250000,74.000000,74.000000,73.250000,78.000000,76.00000,73.500000,72.500000,70.000000
max,200.000000,98.000000,98.000000,99.000000,99.000000,99.000000,99.00000,99.000000,99.000000,99.0000,...,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,98.00000,99.000000,99.000000,99.000000


In [3]:
# wartość optymalizacji po poszczególnym ruchu
optimalization = pd.DataFrame(columns=["Wartosc kombinacji"])
Neighbours_count = 20
improvements = np.zeros(Neighbours_count)
df_backup=df

In [4]:
def E(df=df):
    if len(df) <= 0:
        return 0
    elif len(df) == 1:
        return int(df.sum(axis=1))

    obrobione = df.copy()
    obrobione["M1"] = df["M1"].cumsum() # TODO
    obrobione.iloc[1] = df.iloc[1].cumsum()
    
    for i in range(2, len(obrobione)):
        for j in range(2, len(obrobione.iloc[1])):
            max_val = max(int(obrobione.iloc[[i-1], [j]].values),
                             int(obrobione.iloc[[i], [j-1]].values))
            obrobione.iloc[[i], [j]] = max_val + df.iloc[[i], [j]]
    wartosc = int(obrobione.iloc[[-1], [-1]].values)
    
    return wartosc


In [5]:
# KR to wartość naszego rozwiązania - ma być najmniejsza
#def E(df = df):
 #   c_sum = df["Czas wykonania"].cumsum() # suma kumultatywna kolumny "Czas wykonania"
  #  KR = (df["Termin"] - c_sum)**2 # obliczanie kwadratu różnicy wartosći
   # return sum(KR) # KR jest wskaźnikiem jakości ułożenia zadań. To optymalizujemy

# wyszukiwanie listy sąsiadów
def get_neighbours(idx, neighbours_count = 20, df = df):
    N = neighbours_count
    # Idea sąsiedztwa dla N = 6 i idx = n E <N/2 ; L-N/2>
    # idx = X, sąsiad = 0, nie sąsiad = -
    # nr_indeksu | ... | n-4 | n-3 | n-2 | n-1 |  n  | n+1 | n+2 | n+3 | n+4 | ... |
    # sasiedztwo |  -  |  -  |  0  |  0  |  0  |  X  |  0  |  0  |  0  |  -  |  -  |
    
    L = len(df) - 1 
    idx_prev = idx # indeks dla ktorego szukamy sasiadow
    N_half = N//2 # polowa wartosci N
    
    # Idea sąsiedztwa dla N = 6 i idx = n < N/2
    # idx = X, sąsiad = 0, nie sąsiad = -
    # przykład n = 2
    # nr_indeksu |  0  |  1  | n=2 |  3  |  4  |  5  |  6  |  7  |  8  |  9  | ... |
    # sasiedztwo |  0  |  0  |  X  |  0  |  0  |  0  |  0  |  -  |  -  |  -  |  -  |
    # skoro nie mozemy znaleźć wystarczająco dużo sąsiadów po lewej, 
    # szukamy dodatkowego po prawej
    if idx < N_half:
        idx = N_half
    # idea sąsiedztwa z ograniczeniem z prawej jest analogiczna do tej z ograniczeniem z lewej
    elif idx > L - N_half:
        idx = L - N_half
    output = list(range(idx - N_half, idx + N_half + 1))
    # usuwanie idx z output
    # TODO: na pewno da się to zrobić prościej
    rm_idx = output.index(idx_prev)
    output = np.array(output)
    output = np.delete(output, rm_idx)
    return output

def swap(idx_1, idx_2, df=df):
    # zamiana miejscami dwóch wierszy "in place" - nie tworzymy nowej ramki danych
    df.iloc[idx_1], df.iloc[idx_2] = df.iloc[idx_2].copy(), df.iloc[idx_1].copy()
    
def move(idx_1, idx_2, df=df):
    swap(idx_1, idx_2) # ruch

def reduce_temperature(T, rate_of_change = .02, sposob = 1):
    return (1-rate_of_change)*T if sposob == 1 else T/(1+rate_of_change*T)

def plot_optimalization(optimalization):
    # TODO: można to zrobić ładniej matplotlibem
    optimalization["Wartosc kombinacji"].plot(kind="line")



In [6]:
def simulated_anniling_algorithm(
    T = 2500, 
    Tmin = 100, 
    neighbours_count=20, 
    number_of_generations = 1000
):
    global df
    R = np.random.randint(len(df)) # losowy numer wiersza
    KR_before = E()
    optimalization = pd.DataFrame(columns=["Wartosc kombinacji"])
    optimalization = optimalization.append({
                    "Wartosc kombinacji": KR_before
                }, ignore_index=True)
    while T > Tmin:
        while number_of_generations:
            print(f"{number_of_generations} iterations left")
            neighbours = get_neighbours(idx=R, neighbours_count=neighbours_count) # generuj sąsiadów
            R_prim = np.random.choice(neighbours) # losowy indeks wiersza
            move(R, R_prim)
            KR_now = E()
            dE = KR_now - KR_before
                        # prawdopodobieństwo przyjęcia gorszego rozwiązania
            if dE < 0 or np.random.rand() < np.exp(-dE/T):
                R = R_prim
            KR_before = KR_now
            # dodawanie obecnej wartości kombinacji do listy wszystkich wartości
            optimalization = optimalization.append({
                    "Wartosc kombinacji": KR_now
                }, ignore_index=True)
            number_of_generations -= 1
        T = reduce_temperature(T)
    order = df["Zadanie"]
    return order, optimalization
            

In [ ]:
if __name__ == '__main__':
    new_order, new_optimalization = simulated_anniling_algorithm(
        T=250,
        Tmin=100,
        neighbours_count=6,
        number_of_generations=100
    )
    plot_optimalization(new_optimalization)
    new_optimalization["Wartosc kombinacji"].iloc[-1]

100 iterations left
99 iterations left
98 iterations left
97 iterations left
96 iterations left
95 iterations left
94 iterations left
93 iterations left
92 iterations left
91 iterations left
90 iterations left
89 iterations left
88 iterations left
87 iterations left
86 iterations left
85 iterations left
84 iterations left
83 iterations left
82 iterations left
81 iterations left
80 iterations left
79 iterations left
78 iterations left
77 iterations left
76 iterations left
75 iterations left
74 iterations left
73 iterations left
72 iterations left
71 iterations left
70 iterations left
69 iterations left
68 iterations left
67 iterations left


In [ ]:
plot_optimalization(new_optimalization)

In [ ]:
for order in new_order:
    print(order)

In [ ]:
#pd.to_csv()
order